<a href="https://colab.research.google.com/github/UetaKento/Editer_Like_Emacs_Python/blob/kenty%2FapplyMyRules/Editer_Like_Emacs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# グローバル変数
buffer = ""
cursor = {"row": 0, "column": 0} # rowはカーソルのいる行。 columnはカーソルのいる列。
text = []

# command "a"
def move_cursor_to_head():
    cursor["column"] = 0

# command "e"
def move_cursor_to_end():
    cursor["column"] = len(text[cursor["row"]])-1 # そのrowでの最後尾のindex

# command "p"
def move_cursor_up_and_to_head():
    if is_there_a_char_to("above"):
        move_cursor_by_one_position_to("above")
    move_cursor_to_head()

# command "n"
def move_cursor_down_and_to_head():
    if is_there_a_char_to("below"):
        move_cursor_by_one_position_to("below")
    move_cursor_to_head()

# command "f"
def move_cursor_to_right():
    if is_there_a_char_to("right"):
        move_cursor_by_one_position_to("right")
    elif is_there_a_char_to("below"): # 行末を指しており、かつ下にラインが存在するか確認しているか
        move_cursor_by_one_position_to("below")
        move_cursor_to_head()
    else:
        pass # 行末を指している場合かつ、下に行がない場合は何もしない。

# command "b"
def move_cursor_to_left():
    if is_there_a_char_to("left"):
        move_cursor_by_one_position_to("left")
    elif is_there_a_char_to("above"): # 行頭を指しており、かつ上にラインが存在するか確認しているか
        move_cursor_by_one_position_to("above")
        move_cursor_to_end()
    else:
        pass

# command "d"
def delete_char_on_cursor():
    buf = ""
    if is_there_a_char_on_cursor():
        text[cursor["row"]].pop(cursor["column"]) # 二次元配列から文字列を削除、popのため文字列のシフトは必要ない。
    elif is_there_a_char_to("below"): #行末を指し、下に行があるときは、行末に下の行を連結する。その際に元の行の"\n"は削除
        buf = text.pop(cursor["row"]+1)
        text[cursor["row"]].pop(-1)
        text[cursor["row"]].extend(buf)
    else:
        pass

# command "k"
def delete_with_yanking():
    global buffer # グローバル変数のbufferを参照する
    if is_there_a_char_on_cursor():
        # 以下では「カーソルが指す文字を含めた右側すべての文字を切り取りそれをバッファに記録する。」
        buffer = text[cursor["row"]][cursor["column"]:-1]
        del text[cursor["row"]][cursor["column"]:-1]
        move_cursor_to_end()
    else:
        delete_char_on_cursor()
        buffer = "\n"
    assert buffer != "", "The buffer is empty!"

# command "y"
def paste_buffer():
    buf = []
    if buffer == "":
        pass
    elif buffer == "\n": # バッファの内容が改行の場合、カーソルが指す文字の直前で改行を行う。カーソルは新しい行の先頭文字へ移動する。
        buf = text[cursor["row"]][cursor["column"]:]
        del text[cursor["row"]][cursor["column"]:-1]
        text.insert(cursor["row"]+1, buf)
        move_cursor_by_one_position_to("below")
        move_cursor_to_head()
    else: # バッファの内容が文字列の場合、カーソルの位置にバッファの内容を挿入する。 カーソルの位置はもともと指していた文字または行末の場所へ移動する。
        text[cursor["row"]][cursor["column"]:cursor["column"]] = buffer
        cursor["column"] += len(buffer)

# movementに関する補助関数たち
# カーソルの右または左に文字があるか確認する。
def is_there_a_char_to(direction):
    if direction == "right":
        if cursor["column"] == len(text[cursor["row"]])-1: #カーソルが行末を指していないかを確認
            return False #行末を指す
        else:
            return True # カーソルの右に文字がある
    elif direction == "left":
        if cursor["column"] == 0:
            return False #行頭を指す
        else:
            return True #カーソルの左に文字がある
    elif direction == "above":
        if cursor["row"] == 0:
            return False
        else:
            return True
    elif direction == "below":
        if len(text)-2 == cursor["row"]:
            return False
        else:
            return True

# カーソルを一つ上か下か右か左へ動かす, 遷移先の存在はチェックはしない。
def move_cursor_by_one_position_to(direction):
    if direction == "above":
        cursor["row"] -= 1
    elif direction == "below":
        cursor["row"] += 1
    elif direction == "left":
        cursor["column"] -= 1
    elif direction == "right":
        cursor["column"] += 1
    else:
        pass

# カーソルに文字が存在するかチェック
def is_there_a_char_on_cursor():
    cursor_char = text[cursor["row"]][cursor["column"]]
    if cursor_char != "\n" and cursor_char != "EOF":
        return True
    else:
        return False

# テキスト入力を受け付ける関数
def read_until_eof():
    line = ""
    while True:
        line = input()
        if line == "END_OF_TEXT":
            text.append(["EOF"])
            break
        line = list(line)
        line.append("\n")
        text.append(line)

# コマンドを受けとり、それに対応する関数を実行する関数
def execute_command(command):
    if command == "a":
        move_cursor_to_head()
    elif command == "e":
        move_cursor_to_end()
    elif command == "p":
        move_cursor_up_and_to_head()
    elif command == "n":
        move_cursor_down_and_to_head()
    elif command == "f":
        move_cursor_to_right()
    elif command == "b":
        move_cursor_to_left()
    elif command == "d":
        delete_char_on_cursor()
    elif command == "k":
        delete_with_yanking()
    elif command == "y":
        paste_buffer()
    else:
        pass

def main():
    read_until_eof()

    while True:
        command = input()
        if command == "-":
            break
        execute_command(command)

    while True:
        line = text.pop(0)
        if not line or line[0] == "EOF":
          break
        line.pop(-1) # 改行文字を削除
        line = "".join(line)
        print(line)

if __name__ == "__main__":
    main()

hyo
ni
END_OF_TEXT
f
d
f
f
k
p
p
e
y
s
a
k
y
y
n
y
-
honihoni
honi
